<a href="https://colab.research.google.com/github/gauravcr7rm/cool_name_pending/blob/master/Sentiment_Analysis_Feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/warlordking/sentiment-analysis/archive/master.zip

--2019-03-17 05:08:44--  https://github.com/giapvn/sentiment-analysis/archive/master.zip
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/giapvn/sentiment-analysis/zip/master [following]
--2019-03-17 05:08:44--  https://codeload.github.com/giapvn/sentiment-analysis/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120, 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ] 134.73K  --.-KB/s    in 0.05s   

2019-03-17 05:08:45 (2.56 MB/s) - ‘master.zip’ saved [137962]



In [0]:
!unzip master.zip

Archive:  master.zip
40b28617f82917c356608420e8dcb66d1263afda
   creating: sentiment-analysis-master/
   creating: sentiment-analysis-master/data/
  inflating: sentiment-analysis-master/data/Testing.xml  
  inflating: sentiment-analysis-master/data/Training.xml  
  inflating: sentiment-analysis-master/main.py  


In [0]:
cd sentiment-analysis-master

/content/sentiment-analysis-master


In [0]:
!python main.py

Traceback (most recent call last):
  File "main.py", line 344, in <module>
    main()
  File "main.py", line 227, in main
    train_texts_list, train_opinions_list = load_data(data_directory, "Training.xml")
  File "main.py", line 20, in load_data
    tree = ET.parse(source)
  File "/usr/lib/python3.6/xml/etree/ElementTree.py", line 1196, in parse
    tree.parse(source, parser)
  File "/usr/lib/python3.6/xml/etree/ElementTree.py", line 586, in parse
    source = open(source, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/home/vng/Documents/KD-DM/Project3/sentiment-analysis/data/Training.xml'


In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import os
import xml.etree.ElementTree as ET
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import hstack
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

def load_data(data_directory, filename):
    source = os.path.join(data_directory, filename)
    tree = ET.parse(source)
    root = tree.getroot()
    texts_list = []
    opinions_list = []

    for review in root.findall('Review'):
        text_string = ""
        opinion_inner_list = []

        for sentence in review.findall('sentences/sentence'):
            text_string = text_string + " " + sentence.find('text').text
        texts_list.append(text_string)

        for opinion in review.findall('Opinions/Opinion'):
            opinions_dict = {
                opinion.get('category').replace('#', '_'):opinion.get('polarity')
            }
            opinion_inner_list.append(opinions_dict)
        opinions_list.append(opinion_inner_list)

    return texts_list, opinions_list


def tokenize_user_data(user_input):
    tokens_list  = word_tokenize(user_input.lower())
    return tokens_list


# tokenize sentences into tokens and then save under list format.
def tokenize_data(texts_lists):
    tokens_list = []

    for sentences in texts_lists:
        token_list = word_tokenize(sentences.lower())   # lowercase and then tokenize the sentence
        tokens_list.append(token_list)
    return tokens_list


# trimming data and tokens lemmatization
def clean_data(tokens_list):
    cleaned_tokens_list = []
    lammatizer = nltk.stem.WordNetLemmatizer()

    for tokens in tokens_list:
        # remove punctuation from each token
        table = str.maketrans('', '', punctuation)
        tokens = [word.translate(table) for word in tokens]
        # remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if not word in stop_words]
        # remove remaining tokens that are not alphabetic
        tokens = [word for word in tokens if word.isalpha()]
        # lemmatization
        tokens = [lammatizer.lemmatize(word, 'v') for word in tokens]
        cleaned_tokens_list.append(tokens)

    texts_list = []

    # append tokens in the same review to a text
    for tokens in cleaned_tokens_list:
        text = []
        for token in tokens:
            text.append(token)
        texts_list.append(' '.join(text))

    return texts_list


# select the most common aspects to be labels for Aspect Detection
def get_most_common_aspects(opinions_list):
    opinion = []

    for inner_list in opinions_list:
        for _dict in inner_list:
            for key in _dict:
                opinion.append(key)
    most_common_aspects = [k for k,v in nltk.FreqDist(opinion).most_common(20)] # just get 20 aspects which is the most popular aspect

    return most_common_aspects


# construct a data frame from data after preprocessed
def get_data_frame(texts_list, opinions_list, most_common_aspects):
    data = {'Review': texts_list} # Review column contains list of tokens
    df = pd.DataFrame(data)

    if opinions_list:
        for inner_list in opinions_list:
            for _dict in inner_list:
                for key in _dict:
                    if key in most_common_aspects:
                        df.loc[opinions_list.index(inner_list), key] = _dict[key]

    return df


# construct data frames for #2
# just reach positive aspects
def get_pos_data_frame(df, most_common_aspects):

    for common_aspect in most_common_aspects:
        df[common_aspect] = df[common_aspect].replace(['positive'], [1])
        df[common_aspect] = df[common_aspect].replace(['negative', 'neutral', 'conflict'], [0, 0, 0])
    df = df.fillna(0)

    return df


# just reach negative aspects
def get_neg_data_frame(df, most_common_aspects):

    for common_aspect in most_common_aspects:
        df[common_aspect] = df[common_aspect].replace(['negative'], [1])
        df[common_aspect] = df[common_aspect].replace(['positive', 'neutral', 'conflict'], [0, 0, 0])
    df = df.fillna(0)

    return df


# just reach neutral and conflict aspects
def get_neu_data_frame(df, most_common_aspects):

    for common_aspect in most_common_aspects:
        df[common_aspect] = df[common_aspect].replace(['neutral', 'conflict'], [1, 1])
        df[common_aspect] = df[common_aspect].replace(['negative', 'positive'], [0, 0])
    df = df.fillna(0)

    return df


# normalize the aspect data frame for #1 (aspects mentioned are assigned to 1)
def normalize_aspect_data_frame(df, most_common_aspects):

    for aspect in most_common_aspects:
        df[aspect] = df[aspect].replace(['positive', 'negative', 'neutral', 'conflict'], [1, 1, 1, 1])
    df = df.fillna(0)

    return df


def create_dict_of_aspect(y_train, most_common_aspects):
    positions = [] # contains indexes of aspects implied in review

    for aspects_vector in y_train:
        positions.append([i for i, j in enumerate(aspects_vector) if j == 1]) # i <- index of position; j <- value at this position

    sorted_common_aspects = sorted(most_common_aspects)
    dict_of_aspects = []

    for position in positions:
        inner_dict = {}

        for aspect in sorted_common_aspects:
            if sorted_common_aspects.index(aspect) in position:
                inner_dict[aspect] = 5
            else:
                inner_dict[aspect] = 0
        dict_of_aspects.append(inner_dict)

    return dict_of_aspects


# classify positive, negative, or neutral aspects
def classify_sentiment(df_train, X_train_aspect_dtm, df_test, X_test_aspect_dtm):
    df_train = df_train.reindex_axis(sorted(df_train.columns), axis=1)
    X_train = df_train.Review
    y_train = df_train.drop('Review', 1)
    y_train = np.asarray(y_train, dtype=np.int64)

    df_test = df_test.reindex_axis(sorted(df_test.columns), axis=1)
    X_test = df_test.Review
    y_test = df_test.drop('Review', 1)
    y_test = np.asarray(y_test, dtype=np.int64)

    sentiment_vector = CountVectorizer(stop_words='english', ngram_range=(1, 2))
    X_train_dtm = sentiment_vector.fit_transform(X_train)
    X_test_dtm = sentiment_vector.transform(X_test)

    # combining original and extra train data
    X_train_dtm = hstack((X_train_dtm, X_train_aspect_dtm))
    X_test_dtm = hstack((X_test_dtm, X_test_aspect_dtm))

    nb_classifier = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
    y_pred_class_nb = nb_classifier.predict(X_test_dtm)

    print(metrics.accuracy_score(y_test, y_pred_class_nb))
    print(metrics.classification_report(y_test, y_pred_class_nb))

    rf_classifier = OneVsRestClassifier(RandomForestClassifier()).fit(X_train_dtm, y_train)
    y_pred_class_rf = rf_classifier.predict(X_test_dtm)

    print(metrics.accuracy_score(y_test, y_pred_class_rf))
    print(metrics.classification_report(y_test, y_pred_class_rf))

    svm_classifier = OneVsRestClassifier(svm.SVC()).fit(X_train_dtm, y_train)
    y_pred_class_svm = svm_classifier.predict(X_test_dtm)
    print(metrics.accuracy_score(y_test, y_pred_class_svm))
    print(metrics.classification_report(y_test, y_pred_class_svm))

    return nb_classifier, rf_classifier, svm_classifier






In [0]:
ROOT_PATH = "/content/sentiment-analysis-master"
data_directory = os.path.join(ROOT_PATH, "data")

# load data for pre-processing
train_texts_list, train_opinions_list = load_data(data_directory, "Training.xml")
test_texts_list, test_opinions_list = load_data(data_directory, "Testing.xml")

# pre-process training and testing data
train_tokens_list = tokenize_data(train_texts_list)
train_texts_list = clean_data(train_tokens_list)

test_tokens_list = tokenize_data(test_texts_list)
test_texts_list = clean_data(test_tokens_list)

# get 20 most common aspects
most_common_aspects = get_most_common_aspects(train_opinions_list)
wr = open("./data/common-aspects.txt", "w")
for aspect in most_common_aspects:
  wr.write(aspect + "\n")
wr.close()

#1. Aspect Detection
# convert data format to be corresponding with fit_transform() method
df_train = get_data_frame(train_texts_list, train_opinions_list, most_common_aspects)
df_train.to_csv("./data/training-features.csv")
df = normalize_aspect_data_frame(df_train, most_common_aspects) # aspects mentioned are assigned to 1
df_train_aspect = df.reindex_axis(sorted(df.columns), axis=1) # re-arrange columns in df
X_train = df_train_aspect.Review # split Review column of df to X_train set
y_train = df_train_aspect.drop('Review', 1) # the rest of df is y_train
y_train = np.asarray(y_train, dtype=np.int64)

df_test = get_data_frame(test_texts_list, test_opinions_list, most_common_aspects)
df_test.to_csv("./data/testing-features.csv")
df = normalize_aspect_data_frame(df_test, most_common_aspects)
df_test_aspect = df.reindex_axis(sorted(df.columns), axis=1)
X_test = df_test_aspect.Review
y_test = df_test_aspect.drop('Review', 1)
y_test = np.asarray(y_test, dtype=np.int64)

vect = CountVectorizer(lowercase=False, max_df=1.0, stop_words='english', max_features=2000)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

nb_classifier = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
y_predicted_nb = nb_classifier.predict(X_test_dtm)
#    print(metrics.accuracy_score(y_test, y_predicted_nb))
#    print(metrics.classification_report(y_test, y_predicted_nb))
#    wr = open("./data/predicted-aspects.txt", "w")
#    for aspect in y_predicted_nb:
#        wr.write(str(aspect) + "\n")
#    wr.close()

rf_classifier = OneVsRestClassifier(RandomForestClassifier()).fit(X_train_dtm, y_train)
y_predicted_rf = rf_classifier.predict(X_test_dtm)

#    print(metrics.accuracy_score(y_test, y_predicted_rf))
#    print(metrics.classification_report(y_test, y_predicted_rf))

svm_classifier = OneVsRestClassifier(svm.SVC(C=1.0, kernel='linear')).fit(X_train_dtm, y_train)
y_predicted_svm = svm_classifier.predict(X_test_dtm)
#    print(metrics.classification_report(y_test, y_predicted_svm))
#2. Sentiment Detection for each Aspect
# construct extra data
dict_of_aspects = create_dict_of_aspect(y_train, most_common_aspects)
aspect_vectorizer = DictVectorizer()
X_train_aspect_dtm = aspect_vectorizer.fit_transform(dict_of_aspects)

dict_of_aspects = create_dict_of_aspect(y_test, most_common_aspects)
X_test_aspect_dtm = aspect_vectorizer.transform(dict_of_aspects)

# construct original train data
df_train = get_data_frame(train_texts_list, train_opinions_list, most_common_aspects)
df_test = get_data_frame(test_texts_list, test_opinions_list, most_common_aspects)
# for positive aspect detection
df_train_pos = get_pos_data_frame(df_train, most_common_aspects)
df_test_pos = get_pos_data_frame(df_test, most_common_aspects)
# for negative aspect detection
df_train = get_data_frame(train_texts_list, train_opinions_list, most_common_aspects)
df_test = get_data_frame(test_texts_list, test_opinions_list, most_common_aspects)
df_train_neg = get_neg_data_frame(df_train, most_common_aspects)
df_test_neg = get_neg_data_frame(df_test, most_common_aspects)
# for neutral or conflict aspect detection
df_train = get_data_frame(train_texts_list, train_opinions_list, most_common_aspects)
df_test = get_data_frame(test_texts_list, test_opinions_list, most_common_aspects)
df_train_neu = get_neu_data_frame(df_train, most_common_aspects)
df_test_neu = get_neu_data_frame(df_test, most_common_aspects)

nb_pos_aspect_classifier, rf_pos_aspect_classifier, svm_pos_aspect_classifier = classify_sentiment(df_train_pos, X_train_aspect_dtm, df_test_pos, X_test_aspect_dtm)
nb_neg_aspect_classifier, rf_neg_aspect_classifier, svm_neg_aspect_classifier = classify_sentiment(df_train_neg, X_train_aspect_dtm, df_test_neg, X_test_aspect_dtm)
nb_neu_aspect_classifier, rf_neu_aspect_classifier, svm_neu_aspect_classifier = classify_sentiment(df_train_neu, X_train_aspect_dtm, df_test_neu, X_test_aspect_dtm)


# For an input from keyboard
#    user_input = input("Enter a laptop review: \n")
#    user_tokens = tokenize_user_data(user_input)
#    user_tokens_list = []
#    user_tokens_list.append(user_tokens)
#    user_texts_list = clean_data(user_tokens_list)
#    user_input = pd.Series(user_texts_list)
#    user_input_dtm = vect.transform(user_input)

#    predicted_aspects_nb = nb_classifier.predict(user_input_dtm)
#    predicted_aspects_rf = rf_classifier.predict(user_input_dtm)
#    predicted_aspects_svm = svm_classifier.predict(user_input_dtm)
#    print(str(predicted_aspects_nb))
#    print(str(predicted_aspects_rf))
#    print(str(predicted_aspects_svm))

#    extra_features = create_dict_of_aspect(predicted_aspects_svm, most_common_aspects)
#    extra_features_dtm = DictVectorizer().fit_transform(extra_features)

#    print(user_input_dtm)
#    print(extra_features_dtm)
#    user_input_aspect_dtm = hstack((user_input_dtm, extra_features_dtm))
#    print(user_input_aspect_dtm)
#    pos_aspect = svm_pos_aspect_classifier.predict(user_input_aspect_dtm)
#    print(pos_aspect)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to

0.1375
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        12
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         4
           7       1.00      0.08      0.15        25
           8       0.77      0.95      0.85        58
           9       0.00      0.00      0.00        12
          10       0.93      0.41      0.57        32
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00        10
          13       0.00      0.00      0.00        16
          14       1.00      0.05      0.10        20
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         6
          17       0

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

0.1625
              precision    recall  f1-score   support

           0       0.80      0.73      0.76        11
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        12
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         4
           7       0.88      0.28      0.42        25
           8       0.78      0.78      0.78        58
           9       1.00      0.08      0.15        12
          10       0.73      0.25      0.37        32
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00        10
          13       0.00      0.00      0.00        16
          14       1.00      0.15      0.26        20
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         6
          17       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:184: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:189: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels 

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/s

0.6125
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         8
           8       0.00      0.00      0.00         3
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         0
          17       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/s

0.6125
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         8
           8       0.00      0.00      0.00         3
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         0
          17       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/s